In [1]:
import time
from datetime import datetime
import os
import sys
import numpy as np
import matplotlib
%matplotlib ipympl
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import style

style.use('ggplot')

import multiprocessing
from multiprocessing import Pool

from pathlib import Path

import photutils
import astropy.units as u
from astropy import stats
from astropy.io import fits
from astropy.table import Table, hstack, vstack

from astropy.modeling.models import Gaussian2D, Polynomial2D, Moffat2D
from astropy.modeling.fitting import LevMarLSQFitter

from mmtwfs.wfs import *
from mmtwfs.zernike import ZernikeVector
from mmtwfs.telescope import MMT

In [2]:
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

In [3]:
hdr_keys = ['DATE-OBS', 'RA', 'DEC', 'AZ', 'EL', 'ROT', 'TEMP', 'FOCUS', 'CHAM_DPT', 'CHAM_T', 'OUT_T', 'WIND_W', 'WDIR_W', 'PRESSURE',
            'EXPTIME', 'AIRMASS', 'OSSTEMP', 'TRANSX', 'TRANSY', 'TILTX', 'TILTY']
f5_hdr_keys = ['DATEOBS', 'UT', 'RA', 'DEC', 'AZ', 'EL', 'ROT', 'FOCUS', 'T_OUT', 'T_CHAM', 'RH_OUT', 'RH_CHAM', 'P_BARO', 'EXPTIME', 'AIRMASS',
               'WIND', 'WINDDIR', 'OSSTEMP', 'TRANSX', 'TRANSY', 'TILTX', 'TILTY']
old_f9_keys = ['DATE-OBS', 'TIME-OBS', 'RA', 'DEC', 'AZ', 'EL', 'ROT', 'FOCUS', 'T_OUT', 'T_CHAM', 'RH_OUT', 'RH_CHAM', 'P_BARO', 'EXPTIME', 'AIRMASS',
               'WIND', 'WINDDIR', 'OSSTEMP', 'TRANSX', 'TRANSY', 'TILTX', 'TILTY']

In [4]:
def check_image(f, wfskey=None):
    hdr = {}
    with fits.open(f) as hdulist:
        for h in hdulist:
            hdr.update(h.header)
        data = hdulist[-1].data
    
    # if wfskey is None, figure out which WFS from the header info...
    if wfskey is None:
        # check for MMIRS
        if 'WFSNAME' in hdr:
            if 'mmirs' in hdr['WFSNAME']:
                wfskey = 'mmirs'
        if 'mmirs' in f.name:
            wfskey = 'mmirs'

        # check for binospec
        if 'bino' in f.name:
            wfskey = 'binospec'
        if 'ORIGIN' in hdr:
            if 'Binospec' in hdr['ORIGIN']:
                wfskey = 'binospec'

        # check for new F/9
        if 'f9wfs' in f.name:
            wfskey = 'newf9'
        if 'OBSERVER' in hdr:
            if 'F/9 WFS' in hdr['OBSERVER']:
                wfskey = 'newf9'
        if wfskey is None and 'CAMERA' in hdr:
            if 'F/9 WFS' in hdr['CAMERA']:
                wfskey = 'newf9'

        # check for old F/9
        if 'INSTRUME' in hdr:
            if 'Apogee' in hdr['INSTRUME']:
                wfskey = 'oldf9'
        if 'DETECTOR' in hdr:
            if 'Apogee' in hdr['DETECTOR']:
                wfskey = 'oldf9'

        # check for F/5 (hecto)
        if wfskey is None and 'SEC' in hdr:  # mmirs has SEC in header as well and is caught above
            if 'F5' in hdr['SEC']:
                wfskey = 'f5'

    if wfskey is None:
        # if wfskey is still None at this point, whinge.
        print(f"Can't determine WFS for {f.name}...")

    if 'AIRMASS' not in hdr:
        hdr['AIRMASS'] = np.nan

    # we need to fix the headers in all cases to have a proper DATE-OBS entry with
    # properly formatted FITS timestamp.  in the meantime, this hack gets us what we need 
    # for analysis in pandas.
    dtime = None
    if 'DATEOBS' in hdr:
        dateobs = hdr['DATEOBS']
        if 'UT' in hdr:
            ut = hdr['UT']
        elif 'TIME-OBS' in hdr:
            ut = hdr['TIME-OBS']
        else:
            ut = "07:00:00"  # midnight
        timestring = dateobs + " " + ut + " UTC"
        if wfskey in ('newf9', 'f5'):
            dtime = datetime.strptime(timestring, "%a %b %d %Y %H:%M:%S %Z")
        else:
            dtime = datetime.strptime(timestring, "%Y-%m-%d %H:%M:%S %Z")
    else:
        if wfskey == "oldf9":
            d = hdr['DATE-OBS']
            day, month, year = d.split('/')
            year = str(int(year) + 1900)
            timestring = year + "-" + month + "-" + day + " " + hdr['TIME-OBS'] + " UTC"
            dtime = datetime.strptime(timestring, "%Y-%m-%d %H:%M:%S %Z")
        else:
            timestring = hdr['DATE-OBS'] + " UTC"
            try:
                dtime = datetime.strptime(timestring, "%Y-%m-%dT%H:%M:%S.%f %Z")
            except:
                dtime = datetime.strptime(timestring, "%Y-%m-%dT%H:%M:%S %Z")

    if dtime is None:
        print(f"No valid timestamp in header for {f.name}...")
        obstime = None
    else:
        obstime = dtime.isoformat()
        
    hdr['WFSKEY'] = wfskey
    hdr['OBS-TIME'] = obstime
    return data, hdr

In [5]:
#%cd /Users/tim/MMT/wfsdat/20180209
%cd /Volumes/LACIE\ SHARE/wfsdat/20030321

/Volumes/LACIE SHARE/wfsdat/20030321


In [6]:
file = Path("wfs_start_0188.fits")
data, hdr = check_image(file)
data.shape, hdr

((512, 512),
 {'SIMPLE': True,
  'BITPIX': 16,
  'NAXIS': 2,
  'NAXIS1': 512,
  'NAXIS2': 512,
  'BZERO': 32768,
  'BSCALE': 1,
  'OFFSET1': 0,
  'OFFSET2': 0,
  'XFACTOR': 1,
  'YFACTOR': 1,
  'EXPTIME': 20,
  'ORIGIN': 'Clear Sky Institute',
  'TELESCOP': 'Engineering model',
  'CDELT1': -0.0003444444444,
  'CDELT2': -0.0003444444444,
  'INSTRUME': 'Apogee CCD Camera',
  'JD': 2452720.70917,
  'DATE-OBS': '22/03/103',
  'TIME-OBS': '05:01:12',
  'FILTER': 'C',
  'CAMTEMP': -19,
  'AIRMASS': nan,
  'WFSKEY': 'oldf9',
  'OBS-TIME': '2003-03-22T05:01:12'})

In [36]:
def process_image(f, clobber=False):
    tablefile = Path(f.stem + ".csv")
    if tablefile.exists() and not clobber:
        print(f"{file.name} already processed...")
        return
    else:
        print(f"Processing {file.name}...")

    data, hdr = check_image(file)
    mean, median, stddev = stats.sigma_clipped_stats(data, sigma=3.0, iters=None)
    data = data - median
    spots, fig = wfsfind(data, plot=False)

    apsize = 12.
    apers = photutils.CircularAperture(
            (spots['xcentroid'], spots['ycentroid']),
            r=apsize
    )
    masks = apers.to_mask(method="subpixel")

    props = []
    spot_lines = []
    fit_lines = []
    for m, s in zip(masks, spots):
        tline = {}
        subim = m.cutout(data)
        props_table = photutils.data_properties(subim).to_table()
        props.append(props_table)
        spot_lines.append(s)
        tline['filename'] = file.name
        for k in hdr:
            tline[k] = hdr[k]

        y, x = np.mgrid[:subim.shape[0], :subim.shape[1]]
        sigma = (props_table['semimajor_axis_sigma'][0].value + props_table['semiminor_axis_sigma'][0].value) / 2.

        fitter = LevMarLSQFitter()
        gauss_model = Gaussian2D(
            amplitude=subim.max(),
            x_mean=subim.shape[1]/2.,
            y_mean=subim.shape[0]/2.,
            x_stddev = sigma,
            y_stddev = sigma
        ) + Polynomial2D(degree=0)
        moffat_model = Moffat2D(
            amplitude=subim.max(),
            x_0=subim.shape[1]/2.,
            y_0=subim.shape[0]/2.,
            gamma=sigma
        ) + Polynomial2D(degree=0)
        gauss_fit = fitter(gauss_model, x, y, subim)
        moffat_fit = fitter(moffat_model, x, y, subim)

        gauss_resid = subim - gauss_fit(x, y)
        moffat_resid = subim - moffat_fit(x, y)

        gamma = moffat_fit.gamma_0.value
        alpha = moffat_fit.alpha_0.value
        moffat_fwhm = np.abs(2. * gamma * np.sqrt(2.**(1./alpha) - 1.))
        gauss_fwhm = 0.5 * (gauss_fit.x_stddev_0.value + gauss_fit.y_stddev_0.value) * stats.gaussian_sigma_to_fwhm
        moment_fwhm = 0.5 * (props_table['semimajor_axis_sigma'][0].value + props_table['semiminor_axis_sigma'][0].value) * stats.gaussian_sigma_to_fwhm

        tline['gauss_x'] = gauss_fit.x_mean_0.value
        tline['gauss_y'] = gauss_fit.y_mean_0.value
        tline['gauss_sigx'] = gauss_fit.x_stddev_0.value
        tline['gauss_sigy'] = gauss_fit.y_stddev_0.value
        tline['gauss_amplitude'] = gauss_fit.amplitude_0.value
        tline['gauss_theta'] = gauss_fit.theta_0.value
        tline['gauss_background'] = gauss_fit.c0_0_1.value
        tline['gauss_rms'] = np.nanstd(gauss_resid)
        tline['moffat_amplitude'] = moffat_fit.amplitude_0.value
        tline['moffat_gamma'] = gamma
        tline['moffat_alpha'] = alpha
        tline['moffat_x'] = moffat_fit.x_0_0.value
        tline['moffat_y'] = moffat_fit.y_0_0.value
        tline['moffat_background'] = moffat_fit.c0_0_1.value
        tline['moffat_rms'] = np.nanstd(moffat_resid)
        tline['moment_fwhm'] = moment_fwhm
        tline['gauss_fwhm'] = gauss_fwhm
        tline['moffat_fwhm'] = moffat_fwhm
        fit_lines.append(tline)
    fit_table = Table(fit_lines)
    spot_table = Table(vstack(spot_lines))
    prop_table = Table(vstack(props))
    t = hstack([spot_table, prop_table, fit_table])
    if tablefile.exists():
        if clobber:
            t.write(tablefile, format="csv")
    else:
        t.write(tablefile, format="csv")

In [37]:
process_image(file, clobber=True)

Processing wfs_start_0188.fits...


In [93]:
t.hist(column='gauss_fwhm', bins=50, range=(0,10), alpha=0.6)
t.hist(column='moffat_fwhm', bins=50, range=(0,10), alpha=0.6)
plt.show()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

In [86]:
t['gauss_background'].mean(), t['moffat_background'].mean()

(495.81356566699588, 398.06243108166296)

In [87]:
t.hist(column='ellipticity', bins=50, range=(0,1.0), alpha=0.6)

FigureCanvasNbAgg()

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x1c217115f8>]], dtype=object)

In [97]:
(t['moffat_rms']/t['moffat_amplitude']).mean()

0.015914200328812602

In [91]:
%matplotlib ipympl
plt.imshow(gauss_resids[100])
plt.show()

FigureCanvasNbAgg()

In [22]:
#%matplotlib ipympl
plt.imshow(moffat_resids[10])
plt.show()

In [35]:
%lprun -f process_image process_image(file)

Timer unit: 1e-06 s

Total time: 29.8775 s
File: <ipython-input-33-bb5c1b33ae24>
Function: process_image at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def process_image(f, clobber=False):
     2         1         91.0     91.0      0.0      tablefile = Path(f.stem + ".csv")
     3         1         90.0     90.0      0.0      if tablefile.exists() and not clobber:
     4                                                   print(f"{file.name} already processed...")
     5                                                   return
     6                                           
     7         1     150092.0 150092.0      0.5      data, hdr = check_image(file)
     8         1     510791.0 510791.0      1.7      mean, median, stddev = stats.sigma_clipped_stats(data, sigma=3.0, iters=None)
     9         1        272.0    272.0      0.0      data = data - median
    10         1     322845.0 322845.0      1.1      s